In [1]:
import copy
import torch
import torch.nn as nn
import numpy as np
import random
import os
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn import metrics

from sklearn.decomposition import PCA

In [2]:
import time
import pickle

from math import isnan

# Testing Basic PyTorch and SKLearn Linear Regression

In [3]:
with open(r"C:\Users\kdmen\Desktop\Research\personalization-privacy-risk\Data\continuous_full_data_block1.pickle", 'rb') as handle:
    #refs_block1, poss_block1, dec_vels_block1, int_vel_block1, emgs_block1, Ws_block1, Hs_block1, alphas_block1, pDs_block1, times_block1, conditions_block1 = pickle.load(handle)
    refs_block1, _, _, _, emgs_block1, _, _, _, _, _, _ = pickle.load(handle)
    
keys = ['METACPHS_S106', 'METACPHS_S107', 'METACPHS_S108', 'METACPHS_S109', 'METACPHS_S110', 'METACPHS_S111', 'METACPHS_S112', 'METACPHS_S113', 'METACPHS_S114', 'METACPHS_S115', 'METACPHS_S116', 'METACPHS_S117', 'METACPHS_S118', 'METACPHS_S119']
key_to_num = dict()
num_to_key = dict()
for idx, key in enumerate(keys):
    key_to_num[key] = idx
    num_to_key[idx] = key

num_conds = 8
num_channels = 64
num_updates = 19
cphs_starting_update = 10
update_ix = [0,  1200,  2402,  3604,  4806,  6008,  7210,  8412,  9614, 10816, 12018, 13220, 14422, 15624, 16826, 18028, 19230, 20432, 20769]

In [4]:
# our model
model = torch.nn.Linear(64, 2)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.

criterion = torch.nn.MSELoss(reduction='sum')

#optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005)
# Making LR tiny slows it down but it still blows up...

# Training loop
i=10
for epoch in range(10):
    #for i, (x_data, y_data) in enumerate(trainloader):
    x_data = torch.tensor(emgs_block1[keys[0]][0, update_ix[i]:update_ix[i+1], :], dtype=torch.float32)
    y_data = torch.tensor(refs_block1[keys[0]][0, update_ix[i]:update_ix[i+1], :], dtype=torch.float32)
    # 1) Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x_data)

    # 2) Compute and print loss
    loss = criterion(y_pred, y_data)
    #assert(not isnan(loss))
    if isnan(loss):
        print("LOSS IS NAN")
        break
    print(f'Epoch: {epoch} | Update {i} | Loss: {loss.item()} ')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 0 | Update 10 | Loss: 846854.8125 
Epoch: 1 | Update 10 | Loss: 5991360729448448.0 
Epoch: 2 | Update 10 | Loss: 1.1078499107287658e+27 
Epoch: 3 | Update 10 | Loss: 2.0548135536591992e+38 
Epoch: 4 | Update 10 | Loss: inf 
Epoch: 5 | Update 10 | Loss: inf 
Epoch: 6 | Update 10 | Loss: inf 
Epoch: 7 | Update 10 | Loss: inf 
LOSS IS NAN


In [5]:
# our model
model = torch.nn.Linear(64, 2)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.

criterion = torch.nn.MSELoss(reduction='sum')

#optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005)
# Making LR tiny slows it down but it still blows up...

# Training loop
for epoch in range(10):
    #for i, (x_data, y_data) in enumerate(trainloader):
    for i in range(10, 16):
        x_data = torch.tensor(emgs_block1[keys[0]][0, update_ix[i]:update_ix[i+1], :], dtype=torch.float32)
        y_data = torch.tensor(refs_block1[keys[0]][0, update_ix[i]:update_ix[i+1], :], dtype=torch.float32)
        # 1) Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x_data)

        # 2) Compute and print loss
        loss = criterion(y_pred, y_data)
        #assert(not isnan(loss))
        if isnan(loss):
            print("LOSS IS NAN")
            break
        print(f'Epoch: {epoch} | Update {i} | Loss: {loss.item()} ')

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch: 0 | Update 10 | Loss: 981793.75 
Epoch: 0 | Update 11 | Loss: 6.974981362050662e+16 
Epoch: 0 | Update 12 | Loss: 2.0625084368477386e+28 
Epoch: 0 | Update 13 | Loss: inf 
Epoch: 0 | Update 14 | Loss: inf 
Epoch: 0 | Update 15 | Loss: inf 
Epoch: 1 | Update 10 | Loss: inf 
Epoch: 1 | Update 11 | Loss: inf 
LOSS IS NAN
LOSS IS NAN
LOSS IS NAN
LOSS IS NAN
LOSS IS NAN
LOSS IS NAN
LOSS IS NAN
LOSS IS NAN
LOSS IS NAN


In [6]:
#refs_block1, _, _, _, emgs_block1, _, _, _, _, _, _ = pickle.load(handle)
print(emgs_block1[keys[0]].shape)
print(refs_block1[keys[0]].shape)

(8, 20770, 64)
(8, 20770, 2)


In [7]:
cli0_cond1_samples = emgs_block1[keys[0]][0, update_ix[10]:update_ix[11], :]
cli0_cond1_labels = refs_block1[keys[0]][0, update_ix[10]:update_ix[11], :]

In [8]:
# our model
model = torch.nn.Linear(64, 2)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.

criterion = torch.nn.MSELoss(reduction='sum')

#optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=0.00001)
# Making LR tiny slows it down but it still blows up...

x_data = torch.tensor(cli0_cond1_samples, dtype=torch.float32)
y_data = torch.tensor(cli0_cond1_labels, dtype=torch.float32)

# Training loop
for epoch in range(10):
    #for i, (x_data, y_data) in enumerate(trainloader):
    
    # 1) Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x_data)

    # 2) Compute and print loss
    loss = criterion(y_pred, y_data)
    print(f'Epoch: {epoch} | Loss: {loss.item()} ')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 0 | Loss: 925955.875 
Epoch: 1 | Loss: 37267746816.0 
Epoch: 2 | Loss: 2.7463668010582016e+16 
Epoch: 3 | Loss: 2.0328354753424386e+22 
Epoch: 4 | Loss: 1.504692354803238e+28 
Epoch: 5 | Loss: 1.1137645022367345e+34 
Epoch: 6 | Loss: inf 
Epoch: 7 | Loss: inf 
Epoch: 8 | Loss: inf 
Epoch: 9 | Loss: inf 


In [9]:
for i in range(len(keys)):
    print(f"CLIENT{i}")

    # our model
    model = torch.nn.Linear(64, 2)

    # Construct our loss function and an Optimizer. The call to model.parameters()
    # in the SGD constructor will contain the learnable parameters of the two
    # nn.Linear modules which are members of the model.

    criterion = torch.nn.MSELoss(reduction='sum')

    #optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.00001)
    # Making LR tiny slows it down but it still blows up...

    x_data = torch.tensor(emgs_block1[keys[i]][0, update_ix[10]:, :], dtype=torch.float32)
    y_data = torch.tensor(refs_block1[keys[i]][0, update_ix[10]:, :], dtype=torch.float32)

    # Training loop
    for epoch in range(10):
        #for i, (x_data, y_data) in enumerate(trainloader):

        # 1) Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x_data)

        # 2) Compute and print loss
        loss = criterion(y_pred, y_data)
        print(f'Epoch: {epoch} | Loss: {loss.item()} ')

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print()

CLIENT0
Epoch: 0 | Loss: 6649950.0 
Epoch: 1 | Loss: 9630768955392.0 
Epoch: 2 | Loss: 4.376021094176391e+20 
Epoch: 3 | Loss: 2.006978837730647e+28 
Epoch: 4 | Loss: 9.205255580469576e+35 
Epoch: 5 | Loss: inf 
Epoch: 6 | Loss: inf 
Epoch: 7 | Loss: inf 
Epoch: 8 | Loss: inf 
Epoch: 9 | Loss: inf 

CLIENT1
Epoch: 0 | Loss: 45737372.0 
Epoch: 1 | Loss: 1.1320087247270707e+17 
Epoch: 2 | Loss: 3.515446562205591e+26 
Epoch: 3 | Loss: 1.091733338919507e+36 
Epoch: 4 | Loss: inf 
Epoch: 5 | Loss: inf 
Epoch: 6 | Loss: inf 
Epoch: 7 | Loss: inf 
Epoch: 8 | Loss: inf 
Epoch: 9 | Loss: nan 

CLIENT2
Epoch: 0 | Loss: 8182515.0 
Epoch: 1 | Loss: 7282063572992.0 
Epoch: 2 | Loss: 3.301708733592738e+19 
Epoch: 3 | Loss: 1.5053941427347781e+26 
Epoch: 4 | Loss: 6.864205753656983e+32 
Epoch: 5 | Loss: inf 
Epoch: 6 | Loss: inf 
Epoch: 7 | Loss: inf 
Epoch: 8 | Loss: inf 
Epoch: 9 | Loss: inf 

CLIENT3
Epoch: 0 | Loss: 40029672.0 
Epoch: 1 | Loss: 3.2161161789046784e+17 
Epoch: 2 | Loss: 3.386749015

I guess try and train an sklearn model on the data? No idea what's going on...

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
len(update_ix)

19

In [12]:
def train_and_eval_sklr(emgs_block1, refs_block1, lowerbound, upperbound, test_bool=True, test_data_lb=17, test_data_ub=18):
    print(f"Lowerbound Update: {lowerbound}, Upperbound Update: {upperbound}")
    x_data = torch.tensor(emgs_block1[keys[0]][0, update_ix[lowerbound]:update_ix[upperbound], :], dtype=torch.float32)
    y_data = torch.tensor(refs_block1[keys[0]][0, update_ix[lowerbound]:update_ix[upperbound], :], dtype=torch.float32)

    reg = LinearRegression().fit(x_data, y_data)
    train_score = reg.score(x_data, y_data)
    print(f"train_score: {train_score:0.4f}")
    
    #reg.predict()
    if test_bool:
        print(f"Testing Lowerbound Update: {test_data_lb}, Testing Upperbound Update: {test_data_ub}")
        x_test = torch.tensor(emgs_block1[keys[0]][0, update_ix[test_data_lb]:update_ix[test_data_ub], :], dtype=torch.float32)
        y_test = torch.tensor(refs_block1[keys[0]][0, update_ix[test_data_lb]:update_ix[test_data_ub], :], dtype=torch.float32)
        test_score = reg.score(x_test, y_test)
        print(f"test_score: {test_score:0.4f}")
    
    print()

In [13]:
for i in range(10,16):
    train_and_eval_sklr(emgs_block1, refs_block1, i, i+1)

Lowerbound Update: 10, Upperbound Update: 11
train_score: 0.5949
Testing Lowerbound Update: 17, Testing Upperbound Update: 18
test_score: -26.1648

Lowerbound Update: 11, Upperbound Update: 12
train_score: 0.4977
Testing Lowerbound Update: 17, Testing Upperbound Update: 18
test_score: -20.1097

Lowerbound Update: 12, Upperbound Update: 13
train_score: 0.5045
Testing Lowerbound Update: 17, Testing Upperbound Update: 18
test_score: -18.8112

Lowerbound Update: 13, Upperbound Update: 14
train_score: 0.5950
Testing Lowerbound Update: 17, Testing Upperbound Update: 18
test_score: -11.2738

Lowerbound Update: 14, Upperbound Update: 15
train_score: 0.5361
Testing Lowerbound Update: 17, Testing Upperbound Update: 18
test_score: -18.9421

Lowerbound Update: 15, Upperbound Update: 16
train_score: 0.7134
Testing Lowerbound Update: 17, Testing Upperbound Update: 18
test_score: -3.2824



In [14]:
for i in range(10,16):
    train_and_eval_sklr(emgs_block1, refs_block1, i, 16)

Lowerbound Update: 10, Upperbound Update: 16
train_score: 0.1865
Testing Lowerbound Update: 17, Testing Upperbound Update: 18
test_score: -5.9243

Lowerbound Update: 11, Upperbound Update: 16
train_score: 0.2028
Testing Lowerbound Update: 17, Testing Upperbound Update: 18
test_score: -5.4903

Lowerbound Update: 12, Upperbound Update: 16
train_score: 0.2829
Testing Lowerbound Update: 17, Testing Upperbound Update: 18
test_score: -8.5428

Lowerbound Update: 13, Upperbound Update: 16
train_score: 0.3880
Testing Lowerbound Update: 17, Testing Upperbound Update: 18
test_score: -6.0467

Lowerbound Update: 14, Upperbound Update: 16
train_score: 0.4773
Testing Lowerbound Update: 17, Testing Upperbound Update: 18
test_score: -5.7632

Lowerbound Update: 15, Upperbound Update: 16
train_score: 0.7134
Testing Lowerbound Update: 17, Testing Upperbound Update: 18
test_score: -3.2824

